In [2]:
import seaborn as sns
import scipy
import skimage
sns.set_style("ticks")
from pathlib import Path

# Get the data 

In [3]:
# in this file is the url to the spreadsheet with the data
# PIGLEG_SPREADSHEET_URL=https://docs.google.com/spreadsheets/d/1G55kXxcJ0...piLaK0snddRU-jzU/edit?usp=sharing
env_paths = list(Path(".").glob("../**/.env.dev"))
print(env_paths)
# find line with PIGLEG_SPREADSHEET_URL and give me the value after the "="
PIGLEG_SPREADSHEET_URL = [line.split("=", 1)[1].strip() for line in open(env_paths[0], "r") if line.startswith("PIGLEG_SPREADSHEET_URL")][0]
PIGLEG_SPREADSHEET_URL = PIGLEG_SPREADSHEET_URL.replace("\"", "")
PIGLEG_SPREADSHEET_URL = PIGLEG_SPREADSHEET_URL.replace("edit?usp=sharing", "export?format=csv")
# PIGLEG_SPREADSHEET_URL

[WindowsPath('../.env.dev')]


In [4]:
import pandas as pd
url = PIGLEG_SPREADSHEET_URL
df = pd.read_csv(url)

In [7]:
df.tail()

,filename,report_url,email,uploaded_at,finished_at,fps,Needle holder length [m],Scissors length [m],Forceps length [m],Needle holder visibility [s],...,Right hand bbox stitch 4 length [pix],annotation_annotation,annotation_stars,annotation_annotator,annotation_updated_at,annotation_annotation_annotation,annotation_annotation_stars,annotation_annotation_annotator,annotation_annotation_updated_at,timestamp
853,test_end_2.mp4,http://localhost:8100//uploader/web_report/946...,mjirik@gapps.zcu.cz,2024-01-07 12:30,2024-01-07 12:39,30.0,NaN,"0,05489300889","0,005669856086",NaN,...,NaN,NaN,NaN,NaN,NaN,00:00:02 úprava next,3.0,NaN,2024-01-09 22:25:41.134416+00:00,2024-01-09 23:25:41
854,test_end_2.mp4,http://localhost:8100//uploader/web_report/946...,mjirik@gapps.zcu.cz,2024-01-07 12:30,2024-01-07 12:39,30.0,NaN,"0,05489300889","0,005669856086",NaN,...,NaN,NaN,NaN,NaN,NaN,00:00:02 úprava next\r\n00:00:00 adasdfas,3.0,NaN,2024-01-09 22:26:36.239937+00:00,2024-01-09 23:26:36
855,pigleg_test_4.mp4,http://localhost:8100//uploader/web_report/c9a...,mjirik@gapps.zcu.cz,2024-01-07 11:57,2024-01-10 00:39,29.0,"0,2932883435",NaN,"0,02466326448","6,266666667",...,NaN,NaN,NaN,NaN,NaN,00:00:03 dfasf,5.0,NaN,2024-01-07 15:46:05.919892+00:00,2024-01-10 01:39:51
856,Bad-03_11_21_B_Hopfe_Johanna_Einzelknopfnaht.mp4,http://suran.kky.zcu.cz:8000//uploader/web_rep...,khalatbari.sadaf@gmail.com,2022-07-07 17:33,2024-01-10 08:52,25.0,"5,688334234","0,3564922504","2,515732186","307,8666667",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
857,Bad-03_11_21_B_Hopfe_Johanna_Einzelknopfnaht.mp4,http://suran.kky.zcu.cz:8000//uploader/web_rep...,khalatbari.sadaf@gmail.com,2022-07-07 17:33,2024-01-10 09:12,25.0,"5,688334234","0,3564922504","2,515732186","307,8666667",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [32]:
# remove all records with filename NA or NaN
dfs = df[df["filename"].notna()]

# take all records with filename containing "Good", "Medium", "Bad", if the filename is the same, take the last one

dfs = dfs[dfs["filename"].str.contains("Good|Medium|Bad")].drop_duplicates(subset=["filename"], keep="last")

In [33]:
# add column "stars_auto" with 0 for "Good", 1 for "Medium", 2 for "Bad"
dfs["stars_auto"] = dfs["filename"].str.contains("Good").astype(int) * 0
dfs["stars_auto"] = dfs["filename"].str.contains("Medium").astype(int) * 1
dfs["stars_auto"] = dfs["filename"].str.contains("Bad").astype(int) * 2
# dfs["filename"].str.contains("Good|Medium|Bad")


In [34]:
len(dfs)

22

In [35]:
dfs.columns

Index(['filename', 'report_url', 'email', 'uploaded_at', 'finished_at', 'fps',
       'Needle holder length [m]', 'Scissors length [m]', 'Forceps length [m]',
       'Needle holder visibility [s]',
       ...
       'annotation_annotation', 'annotation_stars', 'annotation_annotator',
       'annotation_updated_at', 'annotation_annotation_annotation',
       'annotation_annotation_stars', 'annotation_annotation_annotator',
       'annotation_annotation_updated_at', 'timestamp', 'stars_auto'],
      dtype='object', length=385)

### Replace "," with "."

In [44]:
# find the columns where its values contain single "," and digits
dfs.columns[dfs.apply(lambda x: (x.dtype is str) and (x.str.contains(",\d").any()))]
cols = []
for col in dfs.columns:
    try:
        if (dfs[col].str.contains(",\d").any()):
            # convert , to . and change the column type to float
            dfs[col] = dfs[col].str.replace(",", ".").astype(float)
            cols.append(col)
    except:
        pass
# cols

In [45]:
# dfs

In [46]:
# print name of columns containing needle holder
# dfs.columns[dfs.columns.str.contains("Needle holder")]

In [47]:
# dfs[["Needle holder stitch 1 length [m]", "Needle holder length [m]"]]

In [48]:
# If column containing " stitch 1" in its name has empty values, fill them with  the values from the column with the same name but withuou " stitch 1"
for col in dfs.columns[dfs.columns.str.contains(" stitch 1")]:
    dfs[col] = dfs[col].fillna(dfs[col.replace(" stitch 1", "")])

In [49]:
# dfs[["Needle holder stitch 1 length [m]", "Needle holder length [m]"]]


# Find the correlation

In [52]:
# find corelation between stars_auto and other numerical columns and sort them
# dfs.corr()["stars_auto"].sort_values()



In [53]:
# try to find the best columns to predict stars_auto
corr = dfs.corr()["stars_auto"].sort_values(ascending=False).head(10)
corr


C:\Users\Jirik\AppData\Local\Temp\ipykernel_34192\3505776196.py:2: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  corr = dfs.corr()["stars_auto"].sort_values(ascending=False).head(10)


stars_auto                              1.000000
Forceps stitch 1 length [m]             0.397985
qr_data_scene_width_m                   0.336219
qr_data_pix_size                        0.336219
Needle holder area presence [%]         0.319609
Forceps stitch 1 area presence [%]      0.318441
Forceps area presence [%]               0.304575
orig frame_height                       0.292419
Forceps length [m]                      0.277674
Needle holder bbox area presence [%]    0.277623
Name: stars_auto, dtype: float64

# Train predictor

In [56]:
import sklearn
# train a model to predict stars_auto
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
# import SVM
from sklearn.svm import SVR
#train a model to predict stars_auto

columns = list(corr.index)
columns.append("filename")

dfsx = dfs[columns]
print(len(dfsx))
dfsx = dfsx.dropna()
print(len(dfsx))
X = dfsx[corr.index].drop("stars_auto", axis=1)
y = dfsx["stars_auto"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = SVR()
model.fit(X_train, y_train)

# 
# X

22
20


-0.7824458388772826

## Score

In [57]:
model.score(X_test, y_test)

-0.7824458388772826

## Check the předictions

In [59]:
y_test

816    2
842    0
840    0
822    2
Name: stars_auto, dtype: int32

In [60]:
y_pred = model.predict(X_test)
y_pred

array([0.12636363, 0.11648296, 0.03380476, 0.10143237])

In [61]:
model.predict(X_train)


array([0.10069563, 0.11084839, 0.09947057, 0.10949899, 0.10615264,
       0.09285266, 0.12814825, 0.11112511, 0.12869557, 0.09870658,
       0.10005963, 0.10915682, 0.10023484, 0.10139677, 0.10023488,
       0.08063209])

In [62]:
y_train

829    0
826    2
833    0
824    2
843    0
841    0
835    2
823    2
830    0
857    2
825    0
834    2
828    0
832    0
839    0
827    0
Name: stars_auto, dtype: int32

In [63]:
from sklearn.neural_network import MLPClassifier
clf = MLPClassifier(solver='lbfgs', alpha=1e-5,
                    hidden_layer_sizes=(5, 2), random_state=1)
clf.fit(X_train, y_train)
clf.score(X_test, y_test)

0.5

In [64]:
clf.predict(X_train)

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [65]:
y_test

816    2
842    0
840    0
822    2
Name: stars_auto, dtype: int32